In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import xgboost as xgb
import numpy as np 

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from IPython.display import display
from sklearn import tree

In [2]:
# !pip install xgboost


# Exploring "games" csv

In [3]:
#load data
games = pd.read_csv('Data/games.csv', dtype = {'GAME_ID': str, 'HOME_TEAM_ID': str, 'VISITOR_TEAM_ID': str})
games.head(3)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0


In [4]:
games.dtypes

GAME_DATE_EST        object
GAME_ID              object
GAME_STATUS_TEXT     object
HOME_TEAM_ID         object
VISITOR_TEAM_ID      object
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

In [5]:
games.loc[games['HOME_TEAM_ID']==1610612753,:].sort_values(by='GAME_DATE_EST')

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS


In [6]:
games = games.dropna()

In [7]:
games = games.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [8]:
bulls = games.loc[games['HOME_TEAM_ID']==1610612753,:].sort_values(by='GAME_DATE_EST')

In [9]:
bulls = bulls[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'HOME_TEAM_WINS'
]]

In [10]:
bulls.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,HOME_TEAM_WINS


In [11]:
games['HOME_TEAM_ID'].unique()

array(['1610612753', '1610612764', '1610612763', '1610612754',
       '1610612761', '1610612752', '1610612738', '1610612740',
       '1610612760', '1610612743', '1610612756', '1610612745',
       '1610612742', '1610612758', '1610612746', '1610612755',
       '1610612739', '1610612766', '1610612748', '1610612749',
       '1610612750', '1610612762', '1610612737', '1610612751',
       '1610612765', '1610612741', '1610612747', '1610612757',
       '1610612759', '1610612744'], dtype=object)

In [12]:
home_var = {'GAME_DATE_EST': [], 
           'GAME_ID': [],
           'HOME_TEAM_ID': [],
           'FG_PCT_home': [],
           'FT_PCT_home': [],
           'FG3_PCT_home': [], 'AST_home': [], 'REB_home': [],'HOME_TEAM_WINS':[]
           }


home_data = pd.DataFrame(home_var)

for team in games['HOME_TEAM_ID'].unique():
    

    temp_df = games.loc[games['HOME_TEAM_ID']==team,:].sort_values(by='GAME_DATE_EST')

    temp_df = temp_df[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'HOME_TEAM_WINS']]
    
#     print(bulls.rolling(2).mean())
    temp_df['FG_PCTHOME_ROllING']=temp_df['FG_PCT_home'].rolling(2).mean()
    temp_df['FT_PCTHOME_ROllING']=temp_df['FT_PCT_home'].rolling(2).mean()
    temp_df['FG3_PCTHOME_ROllING']=temp_df['FG3_PCT_home'].rolling(2).mean()
    temp_df['AST_home_ROllING']=temp_df['AST_home'].rolling(2).mean()
    temp_df['REB_home_ROllING']=temp_df['REB_home'].rolling(2).mean()
#     print(bulls.head())
    home_data = home_data.append(temp_df)
    
    
home_data = home_data.dropna()
home_data = home_data.drop(columns = ['FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home'])
home_data.head()

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,HOME_TEAM_WINS,FG_PCTHOME_ROllING,FT_PCTHOME_ROllING,FG3_PCTHOME_ROllING,AST_home_ROllING,REB_home_ROllING
16017,2003-11-01,20300028,1610612753,0.0,0.400,0.8050,0.3665,17.0,34.5
16002,2003-11-03,20300041,1610612753,0.0,0.320,0.7885,0.3335,14.5,40.0
15992,2003-11-05,20300054,1610612753,0.0,0.355,0.7575,0.3250,17.0,41.5
15979,2003-11-07,20300069,1610612753,0.0,0.424,0.6190,0.3470,19.0,35.0
15941,2003-11-12,20300105,1610612753,0.0,0.420,0.6025,0.2720,18.5,40.5


In [13]:
away_var = {'GAME_DATE_EST': [], 
           'GAME_ID': [],
           'VISITOR_TEAM_ID': [],
           'FG_PCT_away': [],
           'FT_PCT_away': [],
           'FG3_PCT_away': [], 'AST_away': [], 'REB_away': []
           }


away_data = pd.DataFrame(away_var)

for team in games['VISITOR_TEAM_ID'].unique():
    

    temp_df = games.loc[games['VISITOR_TEAM_ID']==team,:].sort_values(by='GAME_DATE_EST')

    temp_df = temp_df[['GAME_DATE_EST', 'GAME_ID', 'VISITOR_TEAM_ID', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away']]
    
#     print(bulls.rolling(2).mean())
    temp_df['FG_PCTAWAY_ROllING']=temp_df['FG_PCT_away'].rolling(2).mean()
    temp_df['FT_PCTAWAY_ROllING']=temp_df['FT_PCT_away'].rolling(2).mean()
    temp_df['FG3_PCTAWAY_ROllING']=temp_df['FG3_PCT_away'].rolling(2).mean()
    temp_df['AST_away_ROllING']=temp_df['AST_away'].rolling(2).mean()
    temp_df['REB_away_ROllING']=temp_df['REB_away'].rolling(2).mean()
#     print(bulls.head())
    away_data = away_data.append(temp_df)

away_data = away_data.dropna()
away_data = away_data.drop(columns = ['FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away'])
away_data.head()

,GAME_DATE_EST,GAME_ID,VISITOR_TEAM_ID,FG_PCTAWAY_ROllING,FT_PCTAWAY_ROllING,FG3_PCTAWAY_ROllING,AST_away_ROllING,REB_away_ROllING
14663,2004-11-10,20400064,1610612766,0.4440,0.7320,0.3335,22.0,41.0
14648,2004-11-12,20400074,1610612766,0.4220,0.6510,0.3420,23.0,42.0
14593,2004-11-20,20400131,1610612766,0.3825,0.5775,0.3835,18.0,42.0
14583,2004-11-21,20400140,1610612766,0.4155,0.6650,0.5280,21.0,44.0
14543,2004-11-26,20400176,1610612766,0.4280,0.7565,0.4445,24.0,43.5


In [14]:
clean_data = pd.merge(home_data, away_data, how="inner", on=["GAME_DATE_EST", "GAME_ID"])

In [15]:
clean_data = clean_data.drop(columns = ["GAME_DATE_EST"])
clean_data.head()

,GAME_ID,HOME_TEAM_ID,HOME_TEAM_WINS,FG_PCTHOME_ROllING,FT_PCTHOME_ROllING,FG3_PCTHOME_ROllING,AST_home_ROllING,REB_home_ROllING,VISITOR_TEAM_ID,FG_PCTAWAY_ROllING,FT_PCTAWAY_ROllING,FG3_PCTAWAY_ROllING,AST_away_ROllING,REB_away_ROllING
0,20300028,1610612753,0.0,0.400,0.8050,0.3665,17.0,34.5,1610612765,0.4260,0.8105,0.3930,16.0,45.0
1,20300041,1610612753,0.0,0.320,0.7885,0.3335,14.5,40.0,1610612752,0.4055,0.9395,0.0800,21.5,43.0
2,20300054,1610612753,0.0,0.355,0.7575,0.3250,17.0,41.5,1610612741,0.4110,0.8350,0.3525,17.5,39.0
3,20300069,1610612753,0.0,0.424,0.6190,0.3470,19.0,35.0,1610612750,0.4500,0.7775,0.4645,22.0,41.5
4,20300105,1610612753,0.0,0.420,0.6025,0.2720,18.5,40.5,1610612763,0.4400,0.7560,0.4190,20.0,37.0


In [16]:
# Logistical Regression

In [17]:
# Assign X (data) and y (target)
X = clean_data.drop("HOME_TEAM_WINS", axis=1)
y = clean_data["HOME_TEAM_WINS"].values.reshape(-1, 1)
print(X.shape, y.shape)

(23380, 13) (23380, 1)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


X_train.head()

,GAME_ID,HOME_TEAM_ID,FG_PCTHOME_ROllING,FT_PCTHOME_ROllING,FG3_PCTHOME_ROllING,AST_home_ROllING,REB_home_ROllING,VISITOR_TEAM_ID,FG_PCTAWAY_ROllING,FT_PCTAWAY_ROllING,FG3_PCTAWAY_ROllING,AST_away_ROllING,REB_away_ROllING
17038,20301150,1610612737,0.4565,0.8425,0.3185,22.0,44.0,1610612738,0.4035,0.6395,0.3250,17.0,40.0
8299,21500236,1610612756,0.4735,0.7480,0.3850,23.0,37.0,1610612744,0.5480,0.6845,0.5480,33.5,49.5
20996,20300346,1610612757,0.5115,0.6465,0.3910,30.5,40.5,1610612749,0.5125,0.7315,0.3775,28.0,39.5
11193,21100065,1610612746,0.4965,0.6125,0.3585,25.5,34.5,1610612757,0.3885,0.8155,0.2925,20.5,43.0
19712,41000301,1610612741,0.4615,0.8000,0.3810,22.0,41.0,1610612748,0.4570,0.9465,0.3030,10.5,39.0


In [19]:
# Create the scalers using only the training data!
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [20]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# create your model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [22]:
# Fit (train) or model using the training data
classifier.fit(X_train_scaled, y_train)

C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [23]:
# Validate the model using the .score() method
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.7583689763330482
Testing Data Score: 0.7563729683490162


In [24]:
# inputs match up - home team / away team 
# query latest stats from api 
# latest rolling averages calculation
# retrain model 
# input user input - output hometeam wins

In [25]:
# XGBOOST

In [26]:
# create your model
model = XGBClassifier()

In [27]:
y_pred= classifier.predict(X_test_scaled)

In [28]:
y_pred

array([1., 1., 1., ..., 1., 1., 0.])

In [29]:
y_test

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [30]:
(y_pred == y_test).sum()/len(y_pred)

3067.663473053892

In [31]:
# TREE

In [32]:
# Fit (train) using the training data
model.fit(X_train_scaled, y_train)

C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:24:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
# Validate the model using the test data
predictions = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.7421727972626176


In [34]:
target = clean_data["HOME_TEAM_WINS"]
target_names = ["loss", "win"]


In [35]:
data_1 = clean_data.drop("HOME_TEAM_WINS", axis=1)
feature_names = data_1.columns
data_1.head()

,GAME_ID,HOME_TEAM_ID,FG_PCTHOME_ROllING,FT_PCTHOME_ROllING,FG3_PCTHOME_ROllING,AST_home_ROllING,REB_home_ROllING,VISITOR_TEAM_ID,FG_PCTAWAY_ROllING,FT_PCTAWAY_ROllING,FG3_PCTAWAY_ROllING,AST_away_ROllING,REB_away_ROllING
0,20300028,1610612753,0.400,0.8050,0.3665,17.0,34.5,1610612765,0.4260,0.8105,0.3930,16.0,45.0
1,20300041,1610612753,0.320,0.7885,0.3335,14.5,40.0,1610612752,0.4055,0.9395,0.0800,21.5,43.0
2,20300054,1610612753,0.355,0.7575,0.3250,17.0,41.5,1610612741,0.4110,0.8350,0.3525,17.5,39.0
3,20300069,1610612753,0.424,0.6190,0.3470,19.0,35.0,1610612750,0.4500,0.7775,0.4645,22.0,41.5
4,20300105,1610612753,0.420,0.6025,0.2720,18.5,40.5,1610612763,0.4400,0.7560,0.4190,20.0,37.0


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_1, target, random_state=42)

In [37]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.66535500427716

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7524379811804961

In [39]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.15085907037575172, 'FG_PCTAWAY_ROllING'),
 (0.14728605384460766, 'FG_PCTHOME_ROllING'),
 (0.09277479718923175, 'FG3_PCTHOME_ROllING'),
 (0.08720458740369325, 'FG3_PCTAWAY_ROllING'),
 (0.08327323322859956, 'REB_home_ROllING'),
 (0.08312640745943085, 'REB_away_ROllING'),
 (0.07438619868921104, 'FT_PCTHOME_ROllING'),
 (0.07320805998828929, 'FT_PCTAWAY_ROllING'),
 (0.07057218897275336, 'GAME_ID'),
 (0.07012304745732219, 'AST_home_ROllING'),
 (0.06718635539110944, 'AST_away_ROllING'),
 (0.0, 'VISITOR_TEAM_ID'),
 (0.0, 'HOME_TEAM_ID')]

In [40]:
ranking_df = pd.read_csv('Data/ranking.csv')
ranking_df.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612747,0,12020,2020-12-21,West,L.A. Lakers,4,4,0,1.00,2-0,2-0,NaN
1,1610612762,0,12020,2020-12-21,West,Utah,3,3,0,1.00,2-0,1-0,NaN
2,1610612740,0,12020,2020-12-21,West,New Orleans,2,2,0,1.00,1-0,1-0,NaN
3,1610612745,0,12020,2020-12-21,West,Houston,4,3,1,0.75,2-0,1-1,NaN
4,1610612763,0,12020,2020-12-21,West,Memphis,4,3,1,0.75,1-1,2-0,NaN


In [41]:
games.shape

(23421, 21)

In [42]:
ranking_df.shape

(188383, 13)

In [43]:
gameslist= list(games.columns)
rankinglist=["W_PCT"]
gameslist+rankinglist

['GAME_DATE_EST',
 'GAME_ID',
 'GAME_STATUS_TEXT',
 'HOME_TEAM_ID',
 'VISITOR_TEAM_ID',
 'SEASON',
 'TEAM_ID_home',
 'PTS_home',
 'FG_PCT_home',
 'FT_PCT_home',
 'FG3_PCT_home',
 'AST_home',
 'REB_home',
 'TEAM_ID_away',
 'PTS_away',
 'FG_PCT_away',
 'FT_PCT_away',
 'FG3_PCT_away',
 'AST_away',
 'REB_away',
 'HOME_TEAM_WINS',
 'W_PCT']

In [44]:
games = games.merge(ranking_df, left_on=["HOME_TEAM_ID", "GAME_DATE_EST"], right_on=["TEAM_ID", "STANDINGSDATE"])[gameslist+rankinglist]

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [45]:
games = games.rename(columns={"W_PCT": "HOME_W_PCT"})
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0
3,2020-12-18,12000039,Final,1610612754,1610612755,2020,1610612754,107.0,0.371,0.692,...,19.0,45.0,1610612755,113.0,0.533,0.629,0.355,23.0,48.0,0
4,2020-12-18,12000040,Final,1610612761,1610612748,2020,1610612761,105.0,0.380,0.737,...,27.0,37.0,1610612748,117.0,0.534,0.741,0.514,30.0,51.0,0


In [46]:
games = games.merge(ranking_df, left_on=["VISITOR_TEAM_ID", "GAME_DATE_EST"], right_on=["TEAM_ID", "STANDINGSDATE"])[gameslist+rankinglist]

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [47]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0
3,2020-12-18,12000039,Final,1610612754,1610612755,2020,1610612754,107.0,0.371,0.692,...,19.0,45.0,1610612755,113.0,0.533,0.629,0.355,23.0,48.0,0
4,2020-12-18,12000040,Final,1610612761,1610612748,2020,1610612761,105.0,0.380,0.737,...,27.0,37.0,1610612748,117.0,0.534,0.741,0.514,30.0,51.0,0


In [48]:
games = games.rename(columns={"W_PCT": "VISITOR_W_PCT"})
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0
3,2020-12-18,12000039,Final,1610612754,1610612755,2020,1610612754,107.0,0.371,0.692,...,19.0,45.0,1610612755,113.0,0.533,0.629,0.355,23.0,48.0,0
4,2020-12-18,12000040,Final,1610612761,1610612748,2020,1610612761,105.0,0.380,0.737,...,27.0,37.0,1610612748,117.0,0.534,0.741,0.514,30.0,51.0,0


In [49]:
list(games.columns)

['GAME_DATE_EST',
 'GAME_ID',
 'GAME_STATUS_TEXT',
 'HOME_TEAM_ID',
 'VISITOR_TEAM_ID',
 'SEASON',
 'TEAM_ID_home',
 'PTS_home',
 'FG_PCT_home',
 'FT_PCT_home',
 'FG3_PCT_home',
 'AST_home',
 'REB_home',
 'TEAM_ID_away',
 'PTS_away',
 'FG_PCT_away',
 'FT_PCT_away',
 'FG3_PCT_away',
 'AST_away',
 'REB_away',
 'HOME_TEAM_WINS']

In [50]:
selected_columns = games[['PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away','FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'VISITOR_W_PCT' ]]

KeyError: "['VISITOR_W_PCT'] not in index"

In [51]:
data = selected_columns.copy()
data.head()

NameError: name 'selected_columns' is not defined

In [52]:
data.size

NameError: name 'data' is not defined

In [53]:
data.HOME_TEAM_WINS.value_counts()

NameError: name 'data' is not defined

In [54]:
data.HOME_TEAM_WINS.value_counts()[1]/data.shape[0]

NameError: name 'data' is not defined

In [55]:
data_df = data.dropna()

NameError: name 'data' is not defined

In [56]:
data_df = data_df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)


NameError: name 'data_df' is not defined

In [57]:
data_df.head()

NameError: name 'data_df' is not defined

In [58]:
data_df.size

NameError: name 'data_df' is not defined

In [59]:
data_df.dtypes

NameError: name 'data_df' is not defined

In [60]:
# LOGISTIC REGRESSION 

In [61]:
# Assign X (data) and y (target)
X = data_df.drop("HOME_TEAM_WINS", axis=1)
y = data_df["HOME_TEAM_WINS"].values.reshape(-1, 1)
print(X.shape, y.shape)

NameError: name 'data_df' is not defined

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


X_train.head()

,GAME_ID,HOME_TEAM_ID,FG_PCTHOME_ROllING,FT_PCTHOME_ROllING,FG3_PCTHOME_ROllING,AST_home_ROllING,REB_home_ROllING,VISITOR_TEAM_ID,FG_PCTAWAY_ROllING,FT_PCTAWAY_ROllING,FG3_PCTAWAY_ROllING,AST_away_ROllING,REB_away_ROllING
17038,20301150,1610612737,0.4565,0.8425,0.3185,22.0,44.0,1610612738,0.4035,0.6395,0.3250,17.0,40.0
8299,21500236,1610612756,0.4735,0.7480,0.3850,23.0,37.0,1610612744,0.5480,0.6845,0.5480,33.5,49.5
20996,20300346,1610612757,0.5115,0.6465,0.3910,30.5,40.5,1610612749,0.5125,0.7315,0.3775,28.0,39.5
11193,21100065,1610612746,0.4965,0.6125,0.3585,25.5,34.5,1610612757,0.3885,0.8155,0.2925,20.5,43.0
19712,41000301,1610612741,0.4615,0.8000,0.3810,22.0,41.0,1610612748,0.4570,0.9465,0.3030,10.5,39.0


In [63]:
# Create the scalers using only the training data!
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)


In [64]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [65]:
# create your model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [66]:
# Fit (train) or model using the training data
classifier.fit(X_train_scaled, y_train)

C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [67]:
# Validate the model using the .score() method
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.7583689763330482
Testing Data Score: 0.7563729683490162


In [68]:
# XGBOOST

In [69]:
# create your model
model = XGBClassifier()

In [70]:
y_pred= classifier.predict(X_test_scaled)

In [71]:
y_pred

array([1., 1., 1., ..., 1., 1., 0.])

In [72]:
y_test

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [73]:
(y_pred == y_test).sum()/len(y_pred)

3067.663473053892

In [74]:
# TREE

In [75]:
# Fit (train) using the training data
model.fit(X_train_scaled, y_train)

C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\15037\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:25:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [76]:
# Validate the model using the test data
predictions = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.7421727972626176


In [77]:
target = data_df["HOME_TEAM_WINS"]
target_names = ["loss", "win"]



NameError: name 'data_df' is not defined

In [78]:
data_df.head()

NameError: name 'data_df' is not defined

In [79]:
data_1 = data_df.drop("HOME_TEAM_WINS", axis=1)
feature_names = data_1.columns
data_1.head()

NameError: name 'data_df' is not defined

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_1, target, random_state=42)

In [81]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6703165098374679

In [82]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7522668947818648

In [83]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.15040163145765956, 'FG_PCTAWAY_ROllING'),
 (0.14817753266390746, 'FG_PCTHOME_ROllING'),
 (0.09300433850312897, 'FG3_PCTHOME_ROllING'),
 (0.08688268665253347, 'FG3_PCTAWAY_ROllING'),
 (0.08426808037884718, 'REB_home_ROllING'),
 (0.08221807484872422, 'REB_away_ROllING'),
 (0.0739913550669753, 'FT_PCTHOME_ROllING'),
 (0.07316833440352492, 'FT_PCTAWAY_ROllING'),
 (0.07041135797138422, 'GAME_ID'),
 (0.07023180685901494, 'AST_home_ROllING'),
 (0.06724480119429972, 'AST_away_ROllING'),
 (0.0, 'VISITOR_TEAM_ID'),
 (0.0, 'HOME_TEAM_ID')]